# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [6]:
# import libraries
import pandas as pd
import re
import numpy as np
from sqlalchemy import create_engine, Table, Column, MetaData
import sqlite3
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils.multiclass import type_of_target
from sklearn.pipeline import FeatureUnion
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import warnings
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import dill as pickle
import os


In [7]:
nltk.download(["stopwords","punkt","wordnet","names","averaged_perceptron_tagger"])
Unwanted_Words = nltk.corpus.stopwords.words("english")
Unwanted_Words.extend([w.lower() for w in nltk.corpus.names.words()])



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### 2. Write a tokenization function to process your text data

In [8]:
database_filepath = 'DisasterResponse.db'
def load_data(database_filepath):    
    
    """ Loads the SQLite databse from the given database_filepath. Stores the messages data in
    df dataframe and further devides the data into input and target variabls.
    Inputs:
          database_filepath-path where the message data stored
    Returns:
           X- inputs containing messages data for modelling
           Y- labels containing the category of messages
           category- list of all the messages categories
    """
    # Data loading from the database   
    engine = create_engine('sqlite:///'+database_filepath)
    df = pd.read_sql_table('DisasterResponseMessages', engine)
    X = df['message']
    Y = df.iloc[:, 4:]
    category_names = Y.columns.tolist()
   

    return X,Y,category_names
    
    
   

In [15]:
def tokenization(text):
    """
    Clean and tokenize text for modeling. It will replace all non-
    numbers and non-alphabets with a blank space. Next, it will
    split the sentence into word tokens and lemmatized them with Nltk's 
    WordNetLemmatizer(), first using noun as part of speech, then verb.
    Finally, the word tokens will be stemmed with Nltk's PorterStemmer.
    
    INPUTS:
        text - the message to be clean and tokenized
    RETURNS:
        clean_tokens: the list containing the cleaned and tokenized words of the message
    """
    tokens = word_tokenize(text)

    # instantiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    text = re.sub(r"[^a-zA-Z0-9]"," ", text.lower())
    for strings in text:
        tokens = nltk.word_tokenize(strings)
        lemmatizer = WordNetLemmatizer()
        string_tokenized = []
        for tok in tokens:
            clean_tok = lemmatizer.lemmatize(tok, pos='v')
            string_tokenized.append(clean_tok.strip())
            
        return string_tokenized

    

    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X,Y,category = load_data(database_filepath)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenization, stop_words=ENGLISH_STOP_WORDS)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
            



### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)

y_pred_df = pd.DataFrame(y_pred, columns = category)
for i in range(35):
        print(category[i],
          classification_report(y_test.iloc[:,i], y_pred_df.iloc[:,i]))
  

request              precision    recall  f1-score   support

          0       0.83      1.00      0.91      4290
          1       0.00      0.00      0.00       897

avg / total       0.68      0.83      0.75      5187

offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      5160
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      5187

aid_related              precision    recall  f1-score   support

          0       0.60      0.90      0.72      3043
          1       0.49      0.13      0.21      2144

avg / total       0.55      0.59      0.51      5187

medical_help              precision    recall  f1-score   support

          0       0.92      1.00      0.96      4772
          1       0.00      0.00      0.00       415

avg / total       0.85      0.92      0.88      5187

medical_products              precision    recall  f1-score   support

          0       0.95      1.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters = {'clf__estimator__n_estimators': [10],
              'clf__estimator__max_features' : ["auto"],
              'clf__estimator__n_jobs': [4],
              'clf__estimator__min_samples_split': [2]
           
              }
rf_model = GridSearchCV(pipeline,parameters, cv=3, verbose=2)

7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
rf_model.fit(X_train,y_train)
y_pred = rf_model.predict(X_test)
category_names = y_test.columns.tolist()
y_pred_df = pd.DataFrame(y_pred, columns = category_names)
accuracy_scoring = (y_test==y_pred).mean()
for i in range(35):
    print(category_names[i],
          classification_report(y_test.iloc[:,i], y_pred_df.iloc[:,i]),{'Accuracy_Score': accuracy_scoring[i]})
    
    

          

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__n_jobs=4 
[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__n_jobs=4, total=  14.4s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__n_jobs=4 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.6s remaining:    0.0s


[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__n_jobs=4, total=  14.4s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__n_jobs=4 
[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, clf__estimator__n_jobs=4, total=  14.4s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.1min finished


request              precision    recall  f1-score   support

          0       0.83      1.00      0.91      4290
          1       0.00      0.00      0.00       897

avg / total       0.68      0.83      0.75      5187
 {'Accuracy_Score': 0.82706766917293228}
offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      5160
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      5187
 {'Accuracy_Score': 0.99479467900520535}
aid_related              precision    recall  f1-score   support

          0       0.60      0.90      0.72      3043
          1       0.50      0.15      0.22      2144

avg / total       0.56      0.59      0.51      5187
 {'Accuracy_Score': 0.58550221708116446}
medical_help              precision    recall  f1-score   support

          0       0.92      1.00      0.96      4772
          1       0.00      0.00      0.00       415

avg / total       0.85      0.92   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


8. Try improving your model further. Here are a few ideas:
try other machine learning algorithms
 add other features besides the TF-IDF

In [20]:
## using AdaBoostClassifier with 10 estimators
pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(tokenizer=tokenization)),
                ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=10,\
                                                         random_state=42)))
                ])

parameters = {
     'tfidf__ngram_range': ((1, 1),(1,3))
}

ada_model = GridSearchCV(pipeline,parameters, cv=5, verbose=1)
ada_model.fit(X_train,y_train)
y_pred = ada_model.predict(X_test)
category_names = y_test.columns.tolist()
y_pred_df = pd.DataFrame(y_pred, columns = category_names)
accuracy_scores = (y_test==y_pred).mean()
for i in range(35):
    print(category_names[i],
          classification_report(y_test.iloc[:,i], y_pred_df.iloc[:,i]), {'Accuracy_Score': accuracy_scores[i]})



Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.5min finished


request              precision    recall  f1-score   support

          0       0.83      1.00      0.91      4290
          1       0.00      0.00      0.00       897

avg / total       0.68      0.83      0.75      5187
 {'Accuracy_Score': 0.82706766917293228}
offer              precision    recall  f1-score   support

          0       0.99      1.00      1.00      5160
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      5187
 {'Accuracy_Score': 0.99479467900520535}
aid_related              precision    recall  f1-score   support

          0       0.60      0.89      0.72      3043
          1       0.49      0.15      0.23      2144

avg / total       0.55      0.58      0.51      5187
 {'Accuracy_Score': 0.58434547908232115}
medical_help              precision    recall  f1-score   support

          0       0.92      1.00      0.96      4772
          1       0.00      0.00      0.00       415

avg / total       0.85      0.92   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
# Model output evaluation
for p, k in zip(accuracy_scoring,accuracy_scores):
    print(round(p,2)*100, round(k,2)*100)


83.0 83.0
99.0 99.0
59.0 57.99999999999999
92.0 92.0
95.0 95.0
97.0 97.0
98.0 98.0
97.0 97.0
100.0 100.0
94.0 94.0
89.0 89.0
91.0 91.0
98.0 98.0
98.0 98.0
99.0 99.0
97.0 97.0
96.0 96.0
87.0 87.0
93.0 93.0
96.0 96.0
95.0 95.0
98.0 98.0
99.0 99.0
99.0 99.0
99.0 99.0
99.0 99.0
96.0 96.0
73.0 72.0
92.0 92.0
90.0 90.0
99.0 99.0
91.0 91.0
98.0 98.0
95.0 95.0
80.0 80.0


In [22]:
# Output shows that the overall accuracy score of Random Forest Model is 93.099 % which is slightly higher than the Overall Accuracy score of 93.094% produced by the Ada Booster model  and thus we will select the Random Forest Classification model for further process.
print({'Random Forest Overall Accuracy Score is':accuracy_scoring.mean()}) 
print({'Ada Booster Overall Accuracy Score is':accuracy_scores.mean()}) 



{'Random Forest Overall Accuracy Score is': 0.93099782423090671}
{'Ada Booster Overall Accuracy Score is': 0.93094274146905709}


### 9. Export your model as a pickle file

In [23]:
with open('classifier.pkl','wb')as file:
    pickle.dump(rf_model,file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.